In [1]:
import json
import pandas as pd
import numpy as np
import datetime
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.providers.mysql.hooks.mysql import MySqlHook
import seaborn as sns
warehouse_hook = PostgresHook(postgres_conn_id='rds_afsg_ds_prod_postgresql_dwh', schema='afsg_ds_prod_postgresql_dwh')
mifos_hook = MySqlHook(mysql_conn_id='mifos_db', schema='mifostenant-safaricom')
airflow_hook = MySqlHook(mysql_conn_id='mysql_airflow', schema='bloom_pipeline')
middleware_hook = PostgresHook(postgres_conn_id='asante_mfs_middleware', schema='asante_datawarehouse')
jubilee_hook = MySqlHook(mysql_conn_id='mifos_db', schema='mifostenant-jubilee')
solv_hook = MySqlHook(mysql_conn_id='solv_ke', schema='solvke_staging')

/tmp/ipykernel_3559/1888061882.py:8 DeprecationWarning: The "schema" arg has been renamed to "database" as it contained the database name.Please use "database" to set the database name.

/tmp/ipykernel_3559/1888061882.py:11 DeprecationWarning: The "schema" arg has been renamed to "database" as it contained the database name.Please use "database" to set the database name.

In [2]:
dt = pd.read_excel('~/data/Accounts with Null Blances - Investigated 20230706.xlsx')

In [3]:
dt.head()

,CFID,LOAN ID,DEBTOR NAME,IDENTIFICATION,MEMBER NO,ACCOUNT NO,CONTACT(s),EMAIL(s),PRINCIPAL AMOUNT,LOAN AMOUNT,...,LAST PAY AMOUNT,LAST ACTION DATE,NEXT ACTION DATE,EMPLOYERS,SEGMENTS,CONTRACT ID,Balance as per Statement Details File (5th July 2023),In Closed Contracts?,Closure Date,Total Repayments by Closure Date
0,170153,315334,Gauku Samuel Mugambi,35196652,174390,315334,254790449667,NaN,1200,1205.76,...,0.0,2023-07-06 08:24:36,2023-07-07 08:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,170042,315146,Daisy Jerop,201300000020855891,159769,315146,254715507379,NaN,1600,1607.68,...,0.0,2023-07-06 08:09:05,2023-07-07 08:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,170044,315088,David Mbede Mwambacha,11310184,56324,315088,254716393718,NaN,1300,1306.24,...,0.0,2023-07-03 09:28:07,2023-07-06 09:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,169701,314264,Peter Ngigi Wanjiku,26399734,13059,314264,254768021277,NaN,13200,13611.84,...,0.0,2023-07-05 08:41:47,2023-07-07 08:41:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,169530,314231,Samwel Machoni Ongaro,39428343,99262,314231,254115653233,NaN,6100,6290.32,...,0.0,2023-07-04 10:22:33,2023-07-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dt['CONTRACT ID2'] = dt['CONTRACT ID'].apply(lambda x: str(x).split('_')[1] if not pd.isnull(x) else x)

by_cntrct = warehouse_hook.get_pandas_df(
    sql="""select loan_mifos_id, cntrct_id, bloom_version, safaricom_loan_balance, safaricom_loan_balance_date from bloomlive.loans_fact_table_materialized_summary_view where cntrct_id in %(ci)s""",
    parameters={'ci': tuple(dt['CONTRACT ID2'].dropna().tolist())}
)

[2023-07-10 11:27:53,269] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

In [5]:
by_cntrct

,loan_mifos_id,cntrct_id,bloom_version,safaricom_loan_balance,safaricom_loan_balance_date
0,243252,110200000001334244,2,NaN,None
1,194004,110900000001183064,2,NaN,None
2,243208,110700000001336251,2,NaN,None
3,243306,110100000001341117,2,NaN,None
4,153175,111100000001093075,2,NaN,None
...,...,...,...,...,...
474,282652,110200000014837596,2,0.0,2023-06-02
475,301926,111400000029892705,2,NaN,None
476,256808,110700000001354626,2,0.0,2023-05-30 21:49:39
477,293146,110800000022365689,2,0.0,2023-05-06 21:13:38


In [15]:
dt[dt['CONTRACT ID2'].notna()]

,CFID,LOAN ID,DEBTOR NAME,IDENTIFICATION,MEMBER NO,ACCOUNT NO,CONTACT(s),EMAIL(s),PRINCIPAL AMOUNT,LOAN AMOUNT,...,LAST ACTION DATE,NEXT ACTION DATE,EMPLOYERS,SEGMENTS,CONTRACT ID,Balance as per Statement Details File (5th July 2023),In Closed Contracts?,Closure Date,Total Repayments by Closure Date,CONTRACT ID2
44,158925,308433,Boniface Somba Muema,201200000005955534,134077,308433,254705478304,NaN,1200,1205.76,...,2023-07-05 11:54:01,2023-07-12 11:54:00,NaN,NaN,ContractID_111200000045838814,Not in open contracts,NO,Not Available,Not Available,111200000045838814
45,158996,308116,Moses Wekesa Masika,23038784,41262,308116,254720868275,NaN,70000,72184.00,...,2023-07-03 11:53:57,2023-07-06 11:53:00,NaN,NaN,ContractID_110300000044639636,Not in open contracts,NO,Not Available,Not Available,110300000044639636
46,158881,307986,Michael Kinyanjui,200200000004823000,109642,307986,254726635001,NaN,160000,164992.00,...,2023-07-05 12:01:18,2023-07-12 12:01:00,NaN,NaN,ContractID_110200000044037540,Not in open contracts,NO,Not Available,Not Available,110200000044037540
47,158890,307969,Julius Maina Njoroge,200400000004927000,111173,307969,254798378937,NaN,20000,20624.00,...,2023-07-01 11:32:15,2023-07-06 11:32:00,NaN,NaN,ContractID_110400000044017111,Not in open contracts,NO,Not Available,Not Available,110400000044017111
48,158796,307962,Felistus Kasiva Masila,23160853,49371,307962,254721110556,NaN,200000,206240.00,...,2023-07-01 11:32:00,2023-07-06 11:32:00,NaN,NaN,ContractID_110300000043950587,Not in open contracts,NO,Not Available,Not Available,110300000043950587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6558,21148,3769,Eunice Nyaguthi Munene,25376069,13871,3769,254725398897,NaN,15000,18777.22,...,NaN,NaN,NaN,NaN,ContractID_110800000000612416,32325,NO,Not Available,Not Available,110800000000612416
6559,21147,3772,Rachel Mkombola Mkombola,26025703,26006,3772,254721169162,NaN,27000,39572.72,...,NaN,NaN,NaN,NaN,ContractID_110900000000608480,46400,NO,Not Available,Not Available,110900000000608480
6560,21152,3760,Julius Mutahi Mutahi,12946569,14029,3760,254701079029,NaN,80000,119142.74,...,NaN,NaN,NaN,NaN,ContractID_111100000000614355,162400,NO,Not Available,Not Available,111100000000614355
6561,21149,3768,Omondi Otieno Charles,28032692,13067,3768,254717913531,NaN,31000,46415.81,...,NaN,NaN,NaN,NaN,ContractID_111400000000615315,65255,NO,Not Available,Not Available,111400000000615315


In [6]:
dt_1 = dt[~dt['CONTRACT ID2'].isin(by_cntrct['cntrct_id'].tolist())]

In [7]:
dmp_trxns = warehouse_hook.get_pandas_df(
    sql="""select * from bloomlive.transactions_data_dump where is_disbursement and id_loan_cntrct in %(ilc)s""",
    parameters={'ilc': tuple(dt_1['CONTRACT ID2'].dropna().tolist())}
)

[2023-07-10 11:28:22,396] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

In [8]:
dt_2 = dt_1[~dt_1['CONTRACT ID2'].isin(dmp_trxns['id_loan_cntrct'].tolist())]

In [14]:
dt_2.dropna(subset=['CONTRACT ID2'])['TAKEN DATE'].max()

'2022-04-24'

In [2]:
dt = pd.read_excel('~/data/Missing Loan refs May 2023.xlsx')

In [3]:
dt['ContractID2'] = dt['ContractID'].apply(lambda x: str(x).split('_')[1])

In [4]:
lns = warehouse_hook.get_pandas_df(
    sql="""select cntrct_id, loan_mifos_id, bloom_version from bloomlive.loans_fact_table_materialized_summary_view where cntrct_id in %(cntrcts)s""",
    parameters={'cntrcts': tuple(dt['ContractID2'].tolist())}
)

[2023-06-09 14:47:19,344] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

In [15]:
lns.head()

,cntrct_id,loan_mifos_id,bloom_version
